<h1>언어 인지</h1>

In [1]:
# # huggingface 설치: 사전학습 모델 사용을 위한 라이브러리
# !pip install transformers
# !pip install datasets

In [2]:
# # 라이브러리
# import tensorflow as tf
# import numpy as np
# import matplotlib.pyplot as plt

In [3]:
# # IMBD dataset: 영화리뷰 데이터셋
# from datasets import load_dataset
# data = load_dataset('imdb')
# xtrain, ytrain = list(data['train']['text']), list(data['train']['label'])
# xtest, ytest = list(data['train']['text']), list(data['train']['label'])

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset imdb downloaded and prepared to /Users/juneon/.cache/huggingface/datasets/imdb/plain_text/1.0.0/d613c88cf8fa3bab83b4ded3713f1f74830d1100e171db75bbddb80b3345c9c0. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

# 탐색적 데이터 분석

In [4]:
print( len(xtrain), len(xtest) )

25000 25000


In [5]:
xtrain[0]

'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far between, ev

In [6]:
ytrain[0] # negative

0

In [7]:
xtrain[13000]

"Visually stunning and full of Eastern Philosophy, this amazing martial arts fantasy is brought to you by master director Tsui Hark, the man behind some of the best films Hong Kong cinema has produced. The special effects are beautiful and imaginative. The plot is a bit on the cerebral side, but is a refreshing change from films that treat their audience as if they were morons. If thinking is not your forte, however, this may not be your movie. Maybe you should go see the latest from the Hollywood studio's no brain club, but if you are looking for something more, he's where you will find it."

In [8]:
ytrain[13000] # positive

1

In [9]:
np.unique(ytrain,return_counts=True)

(array([0, 1]), array([12500, 12500]))

# 텐서플로우를 이용한 감성분석

텍스트 벡터화 (One-hot Encoding)

In [10]:
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens = 10000,             # number of tokens # 많이 등장한 토큰 상위 10000개
    output_sequence_length = 64    # maximum length of sequence
)
vectorize_layer.adapt(xtrain)

In [11]:
vectorize_layer('I love you')

<tf.Tensor: shape=(64,), dtype=int64, numpy=
array([ 10, 116,  23,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0])>

모델링

In [12]:
model = tf.keras.Sequential()
model.add( vectorize_layer )
model.add( tf.keras.layers.Embedding(10000,32) )  # 10000x32 임베팅 행렬
model.add( tf.keras.layers.SimpleRNN(16) )        # 은닉정보 벡터크기 = 16
model.add( tf.keras.layers.Dense(2,activation='softmax') )  # 문장 분류

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 64)                0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 64, 32)            320000    
                                                                 
 simple_rnn (SimpleRNN)      (None, 16)                784       
                                                                 
 dense (Dense)               (None, 2)                 34        
                                                                 
Total params: 320818 (1.22 MB)
Trainable params: 320818 (1.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


모델 훈련

In [14]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [15]:
model.fit(xtrain,ytrain,epochs=5)

Epoch 1/5
782/782 [==============================] - 4s 5ms/step - loss: 0.5516 - accuracy: 0.7150
Epoch 2/5
782/782 [==============================] - 4s 5ms/step - loss: 0.3773 - accuracy: 0.8399
Epoch 3/5
782/782 [==============================] - 4s 5ms/step - loss: 0.2609 - accuracy: 0.8986
Epoch 4/5
782/782 [==============================] - 4s 5ms/step - loss: 0.1457 - accuracy: 0.9492
Epoch 5/5
782/782 [==============================] - 4s 5ms/step - loss: 0.0796 - accuracy: 0.9741


평가 및 테스트

In [16]:
model.evaluate(xtest,ytest)

782/782 [==============================] - 2s 2ms/step - loss: 0.0542 - accuracy: 0.9858


[0.0542452335357666, 0.98580002784729]

In [17]:
model.predict(['I love this movie', 'Very terrible movie'])

1/1 [==============================] - 0s 81ms/step


array([[8.9427648e-04, 9.9910575e-01],
       [9.9707425e-01, 2.9257454e-03]], dtype=float32)

# 트랜스포머

In [25]:
from transformers import pipeline

문장 분류

In [19]:
classifier = pipeline(task='text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.


In [20]:
classifier(['I love this movie', 'Very terrible movie'])

[{'label': 'POSITIVE', 'score': 0.9998766183853149},
 {'label': 'NEGATIVE', 'score': 0.9996885061264038}]

문장 생성

In [21]:
generator = pipeline(task='text-generation')

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


In [22]:
generator('In this course, I will teach you how to', max_length=30)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'In this course, I will teach you how to write a great program, the most important thing is to know how to do this successfully. By taking'}]

In [49]:
# 한국어 문장 생성
kor_generator = pipeline("text-generation", model="skt/kogpt2-base-v2")


ValueError: Could not load model skt/kogpt2-base-v2 with any of the following classes: (<class 'transformers.models.auto.modeling_tf_auto.TFAutoModelForCausalLM'>, <class 'transformers.models.gpt2.modeling_tf_gpt2.TFGPT2LMHeadModel'>).

In [28]:
kor_generator("오늘 야구 경기에서는 ", max_length=30)

NameError: name 'kor_generator' is not defined

빈칸 채우기

In [29]:
# 빈칸 채우기
unmasker = pipeline("fill-mask")

No model was supplied, defaulted to distilroberta-base and revision ec58a5b (https://huggingface.co/distilroberta-base).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFRobertaForMaskedLM.

All the weights of TFRobertaForMaskedLM were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForMaskedLM for predictions without further training.


In [30]:
unmasker("This course will teach you all about <mask> models.", top_k=2)

[{'score': 0.19619612395763397,
  'token': 30412,
  'token_str': ' mathematical',
  'sequence': 'This course will teach you all about mathematical models.'},
 {'score': 0.04052687808871269,
  'token': 38163,
  'token_str': ' computational',
  'sequence': 'This course will teach you all about computational models.'}]

Q&A

In [31]:
question_answerer = pipeline("question-answering")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


All PyTorch model weights were used when initializing TFDistilBertForQuestionAnswering.

All the weights of TFDistilBertForQuestionAnswering were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForQuestionAnswering for predictions without further training.


In [32]:
question_answerer(
    question="Where do I study?",
    context="My name is Jaewon and I am a student of Korea University"
)

{'score': 0.9710102081298828,
 'start': 40,
 'end': 56,
 'answer': 'Korea University'}

# 연습문제

1. 위의 IMBD 데이터를 학습하는 모델에서 SImpleRNN 모듈 대신에 GRU 모듈을 사용하여 모델을 구성하고 학습하시오.
```tf.keras.layers.GRU```
를 사용하시오. <br>
(1) 모델의 총 파라메터의 수는 몇 개인가? <br>
(2) 학습 후 평가데이터의 정확도는 얼마인가? <br>



In [37]:
model = tf.keras.Sequential()
model.add( vectorize_layer )
model.add( tf.keras.layers.Embedding(10000,32) )  # 10000x32 임베팅 행렬
model.add( tf.keras.layers.GRU(16) )        # 은닉정보 벡터크기 = 16
model.add( tf.keras.layers.Dense(2,activation='softmax') )  # 문장 분류

In [38]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVe  (None, 64)                0         
 ctorization)                                                    
                                                                 
 embedding_1 (Embedding)     (None, 64, 32)            320000    
                                                                 
 gru (GRU)                   (None, 16)                2400      
                                                                 
 dense_1 (Dense)             (None, 2)                 34        
                                                                 
Total params: 322434 (1.23 MB)
Trainable params: 322434 (1.23 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


모델 훈련

In [40]:
model.compile(
    optimizer = 'adam',
    loss = 'sparse_categorical_crossentropy',
    metrics = ['accuracy']
)

In [41]:
model.fit(xtrain,ytrain,epochs=5)

Epoch 1/5
782/782 [==============================] - 7s 8ms/step - loss: 0.5381 - accuracy: 0.7074
Epoch 2/5
782/782 [==============================] - 6s 8ms/step - loss: 0.3626 - accuracy: 0.8432
Epoch 3/5
782/782 [==============================] - 6s 8ms/step - loss: 0.2930 - accuracy: 0.8810
Epoch 4/5
782/782 [==============================] - 6s 8ms/step - loss: 0.2273 - accuracy: 0.9130
Epoch 5/5
782/782 [==============================] - 6s 8ms/step - loss: 0.1662 - accuracy: 0.9405


평가 및 테스트

In [42]:
model.evaluate(xtest,ytest)

782/782 [==============================] - 2s 2ms/step - loss: 0.0961 - accuracy: 0.9713


[0.09609945118427277, 0.9713199734687805]

2. HuggingFace의 pipeline을 사용하여 문장 분류를 수행하시오. 기본적인 모델을 사용하였을 때, "This moive is so so"에 대한 긍정 확률은 얼마인가?

In [43]:
from transformers import pipeline

In [44]:
classifier = pipeline(task='text-classification')

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
All PyTorch model weights were used when initializing TFDistilBertForSequenceClassification.

All the weights of TFDistilBertForSequenceClassification were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertForSequenceClassification for predictions without further training.
